In [ ]:

from IPython.display import Image  # for displaying images
import os 
import random
import numpy as np
import matplotlib.pyplot as plt

from xml.dom import minidom
from tqdm import tqdm
from PIL import Image, ImageDraw

import shutil
from sklearn.model_selection import train_test_split
import xml.etree.ElementTree as ET

import torch

In [ ]:
import xml.etree.ElementTree as ET

def extract_info_from_xml(xml_file_path):
    tree = ET.parse(xml_file_path)
    root = tree.getroot()

    info_dict = {
        'filename': None,
        'image_size': None,
        'bboxes': []
    }

    for elem in root:
        if elem.tag == "filename":
            info_dict['filename'] = elem.text

        elif elem.tag == "size":
            image_size = [int(subelem.text) for subelem in elem]
            info_dict['image_size'] = tuple(image_size)

        elif elem.tag == "object":
            bbox = {}
            for subelem in elem:
                if subelem.tag == "name":
                    bbox["class"] = subelem.text
                elif subelem.tag == "bndbox":
                    for subsubelem in subelem:
                        bbox[subsubelem.tag] = int(subsubelem.text)
            info_dict['bboxes'].append(bbox)

    return info_dict

print(extract_info_from_xml('ArmaCV_dataset/labels/complex (24).xml'))


In [ ]:
import os

# Dictionary that maps class names to IDs
class_name_to_id_mapping = {"marid": 0, "zamak": 1, "complex":2, "varsuk":3 }

# Convert the info dict to the required YOLOv5 format and write it to disk
def convert_to_yolov5(info_dict):
    annotations = []

    # For each bounding box
    for b in info_dict["bboxes"]:
        class_name = b["class"]

        # Check if the class exists in the mapping
        if class_name not in class_name_to_id_mapping:
            print("Invalid Class. Must be one from", list(class_name_to_id_mapping.keys()))
            continue

        class_id = class_name_to_id_mapping[class_name]

        # Transform the bbox co-ordinates as per the format required by YOLOv5
        b_center_x = (b["xmin"] + b["xmax"]) / 2
        b_center_y = (b["ymin"] + b["ymax"]) / 2
        b_width = (b["xmax"] - b["xmin"])
        b_height = (b["ymax"] - b["ymin"])

        # Normalize the co-ordinates by the dimensions of the image
        image_w, image_h, _ = info_dict["image_size"]
        b_center_x /= image_w
        b_center_y /= image_h
        b_width /= image_w
        b_height /= image_h

        # Add the annotation to the list
        annotations.append(f"{class_id} {b_center_x:.3f} {b_center_y:.3f} {b_width:.3f} {b_height:.3f}")

    # Name of the file to save
    save_file_name = os.path.join("./annotations", info_dict["filename"].replace("png", "txt"))

    # Save the annotations to disk
    with open(save_file_name, "w") as file:
        file.write("\n".join(annotations))


In [ ]:
# Get the annotations
annotations = [os.path.join('./annotations/', x) for x in os.listdir('./annotations/') if x[-3:] == "xml"]
annotations.sort()

# Convert and save the annotations
for ann in tqdm(annotations):
    info_dict = extract_info_from_xml(ann)
    convert_to_yolov5(info_dict)
    
annotations = [os.path.join('./annotations/', x) for x in os.listdir('./annotations/') if x[-3:] == "txt"]

In [ ]:
class_name_to_id_mapping = {"marid": 0, "zamak": 1, "complex":2, "varsuk":3 }

annotations = [os.path.join('./', x) for x in os.listdir('./annotations') if x[-3:] == "txt"]
annotations

In [ ]:
class_id_to_name_mapping = dict(zip(class_name_to_id_mapping.values(), class_name_to_id_mapping.keys()))

def plot_bounding_box(image, annotation_list):
    annotations = np.array(annotation_list)
    w, h = image.size
    
    plotted_image = ImageDraw.Draw(image)

    transformed_annotations = np.copy(annotations)
    transformed_annotations = transformed_annotations.reshape(-1, 5)  # Reshape to (num_annotations, num_attributes)
    transformed_annotations[:,[1,3]] = transformed_annotations[:,[1,3]] * w
    transformed_annotations[:,[2,4]] = transformed_annotations[:,[2,4]] * h 
    
    transformed_annotations[:,1] = transformed_annotations[:,1] - (transformed_annotations[:,3] / 2)
    transformed_annotations[:,2] = transformed_annotations[:,2] - (transformed_annotations[:,4] / 2)
    transformed_annotations[:,3] = transformed_annotations[:,1] + transformed_annotations[:,3]
    transformed_annotations[:,4] = transformed_annotations[:,2] + transformed_annotations[:,4]
    
    for ann in transformed_annotations:
        obj_cls, x0, y0, x1, y1 = ann
        plotted_image.rectangle(((x0,y0), (x1,y1)))
        
        plotted_image.text((x0, y0 - 10), class_id_to_name_mapping.get(int(obj_cls), 'Unknown'))
    
    plt.imshow(np.array(image))
    plt.show()

annotations = [x for x in os.listdir('./annotations') if x[-3:] == "txt"]
annotation_file = random.choice(annotations)
with open("./annotations/"+annotation_file, "r") as file:
    annotation_list = file.read().split("\n")[:-1]
    annotation_list = [x.split(" ") for x in annotation_list]
    annotation_list = [[float(y) for y in x ] for x in annotation_list]

print(annotation_file)
#Get the corresponding image file
image_file = "./images/"+annotation_file.replace("txt", "png")
#assert os.path.exists(image_file)

#Load the image
image = Image.open(image_file)

#Plot the Bounding Box
plot_bounding_box(image, annotation_list)

In [ ]:
# Read images and annotations->
images = [os.path.join('./images', x) for x in os.listdir('images')]
annotations = [os.path.join('./annotations', x) for x in os.listdir('annotations') if x[-3:] == "txt"]

images.sort()
annotations.sort()

# train-valid-test splits 
train_images, test_images, train_annotations, test_annotations = train_test_split(images, annotations, test_size = 0.2, random_state = 1)


In [ ]:
shutil.os.makedirs("./images/train/", exist_ok=True)
shutil.os.makedirs("./images/test/", exist_ok=True)
shutil.os.makedirs("./annotations/train/", exist_ok=True)
shutil.os.makedirs("./annotations/test/", exist_ok=True)

#Utility function to move images 
def move_files_to_folder(list_of_files, destination_folder):
    for f in list_of_files:
        try:
            shutil.copy(f, destination_folder)
        except:
            print(f)
            assert False

# Move the splits into their folders
move_files_to_folder(train_images, 'images/train')
move_files_to_folder(test_images, 'images/test/')
move_files_to_folder(train_annotations, 'annotations/train/')
move_files_to_folder(test_annotations, 'annotations/test/')

For recover issues, delete images if there is not any error occured

In [ ]:
import os

folder_path = './images' 

# Iterate through files in the folder
for filename in os.listdir(folder_path):
    if filename.endswith('.png'):
        file_path = os.path.join(folder_path, filename)
        try:
            os.remove(file_path)  # Delete the .png file
            print(f"Deleted: {filename}")
        except OSError as e:
            print(f"Error deleting {filename}: {e}")

print("Done deleting .png files.")

In [ ]:
import os

folder_path = './annotations' 

# Iterate through files in the folder
for filename in os.listdir(folder_path):
    if filename.endswith('.txt'):
        file_path = os.path.join(folder_path, filename)
        try:
            os.remove(file_path)  # Delete the .png file
            print(f"Deleted: {filename}")
        except OSError as e:
            print(f"Error deleting {filename}: {e}")

print("Done deleting .png files.")